# Let Dialyzer Work (For Ease Of Our Mind)

[ymtszw/notes](https://github.com/ymtszw/notes/blob/master/20170913_let_dialyzer_work.ipynb)

## Preface 1

- このドキュメントは試験的に[Jupyter notebook](http://jupyter.org/)で書いている。
- Elixirコードを実行するには[iElixir kernel](https://github.com/pprzetacznik/IElixir)をインストールする必要がある。
    - これを書いている時点ではElixir1.5以降に未対応だが、特にコンパイルできないということもない。
    - 単にmix.exsの`:elixir`バージョン指定を`~> 1.4`等に緩めてインストールすればElixir 1.5系でも動作する。

- きっとLive Presentationするための拡張がなにがしかあるだろうと探したところ、[damianavila/RISE](https://github.com/damianavila/RISE)というのがやっぱりあったので使っている。
- Notebookファイルからslide形式に出力するだけなら、その機能自体はデフォルトの`jupyter-nbconvert`が最初から持っている。
    - `jupyter-nbconvert <file>.ipynb --to slides`
    - reveal.jsに対応したHTMLを吐くので、関連するアセットを自前でサーブしている環境があるなら、置けば動きそう。
    - 自分は用意していなかったのでRISEに頼る。

- ただし、RISEはまだまだ開発途上で、環境としては癖が強いので我慢が必要。
- 特に、Jupyter notebookのデフォルトのキーバインドとreveal.jsのキーバインドとで競合するので、慣れたreveal.jsのキーバインドがまともに動かない。
    - 横方向は←→でいけるが、縦方向はPgUp/PgDnが必要なのでMacBookだとfnキーを使う必要がある。
- とはいえそこそこ使える。プレゼン表示したままcellを編集できるので、なんちゃってWYSIWYG編集もできるし、code cellの実行ももちろん可能。
- 文字サイズとかはブラウザで調整してごまかす。

## Preface 2

- Jupyter notebookのkernelは基本的には各言語の対話環境に相当する。
    - Elixirでは`iex`相当になる。

In [2]:
:erlang.system_info(:otp_release)

'20'

In [4]:
Application.spec(:elixir)[:vsn]

'1.5.1'

- `iex`の中でも複数行コード実行や`defmodule`を使ったmodule生成等ができることは試せば明らかだが、iElixir経由でもできることを確認しておく。
    - 後に説明で使う。
    - `import Kernel`はされているが、コード補完は何かmoduleが前置されていないと動かないようにみえる。

In [14]:
defmodule ModuleDefinedFromJupyter do
  def echo(term) do
    term
  end
      
  def recho(term) when is_binary(term) do
    String.reverse(term)
  end                   
end
alias ModuleDefinedFromJupyter, as: M
M.echo("hello world") <> M.recho("hello world")

"hello worlddlrow olleh"

## Preface 3

- このドキュメントではErlang/Elixirの話をするが、Erlang/Elixirについてのベーシックな説明は省く。
- [dialyzer](http://erlang.org/doc/man/dialyzer.html)に託けて、Elixirでのコーディングについてふわっとした話をしたい。
- いろいろ試したけどiElixir(iex)経由でdialyzerまで実行させながらというのは無理があるのでやめた。
- mix projectを用意して、その中のコードに対してdialyzerを別で実行して結果を貼り付けながら説く。
    - Elixir codeの部分についてはcode cellに書かれているものは実行可能。

- Shellで`mix new`して適当にプロジェクトを作り、mix.exsを編集して以下のdepsを加える。
    - `{:dialyze, "0.2.1", [only: :dev]}`
- 今は[jeremyjh/dialyxir](https://github.com/jeremyjh/dialyxir)の方が使われていると思うが、諸事情でこのドキュメントではディスコンしている[fishcakez/dialyze](https://github.com/fishcakez/dialyze)の方を使う。本質的な違いはない。
    - Task実行は`mix dialyze`
    - Dialyxirなら`mix dialyxir`
    - `dialyzer` CLIを直接使ってもいいのだが、これらのMix taskはErlang/Elixirの標準モジュールやdepsに含まれるモジュールのPLT（後述）生成等のコマンドを含むので、楽。

- Dialyzeするところまでやってみると以下のようになるはず。
- 以下のCode cellは[BashKernel](https://github.com/takluyver/bash_kernel)に切り替えれば実行できるが、今する必要はない。

In [5]:
# Bash
pwd

/Users/yumatsuzawa/workspace/notes/20170913_let_dialyzer_work/dial


In [7]:
# Bash
mix deps.get

Resolving Hex dependencies...
Dependency resolution completed:
  dialyze 0.2.1
* Getting dialyze (Hex package)
  Checking package (https://repo.hex.pm/tarballs/dialyze-0.2.1.tar)
  Using locally cached package


In [10]:
# Bash
# 初回実行時はかなり時間がかかるはず。あと、1.5系ではWarningが出るが気にしない。
mix dialyze

Finding applications for analysis
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 1 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt


- 最後の行が今回のmix projectのmoduleに対するDialyzer実行結果になる
    - 1 module解析し、特に何も問題なかったということ。
- `mix new`したてのElixir applicationには、"application_name.ex"というファイルの中に`ApplicationName`というmoduleがあるだけなので、正しい結果である。サンプルコードの場合は`Dial`がある。一応現状の中身を書いておく。
    - 多分`@moduledoc`が生成されていると思うが、省略。

In [ ]:
defmodule Dial do
  def hello do
    :world
  end
end

## Let Dialyzer Work

- ようやく本題。
- dialyzerはErlang同梱の静的解析ツール。
- 解析に利用するためのapplicationやmoduleに関する情報をまとめたPersistant Lookup Table (PLT) ファイルを生成したのち、"success typing"という不思議な魔法でコードを解析する。

## Success typing

- [Learn You Some Erlang](http://learnyousomeerlang.com/dialyzer)によれば、
> As the [Success Typings paper](http://www.it.uu.se/research/group/hipe/papers/succ_types.pdf) behind Dialyzer explains it, a type checker for a language like Erlang should ... (snip) ... **only complain on type errors that would guarantee a crash.**
- 「絶対におかしい、間違いなく型エラーでクラッシュするときだけ警告する」解析手法である。

## Let it complain

- 試しに何か言わせてみる。
- 大前提として、Erlangは静的型付け言語ではない。以下のようなモジュールはコンパイルできるが、当然実行時例外になる。

In [1]:
defmodule Dial do
  def crash() do
    1 + "2"
  end
end
    
Dial.crash()

ArithmeticError: 1

- Dialyzerはこのようなあからさまなバグを持つコードを警告できるだろうか。

In [5]:
# Bash
mix dialyze

Finding applications for analysis
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 1 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
lib/dial.ex:2: Function crash/0 has no local return

lib/dial.ex:3: The call erlang:'+'(1,<<_:8>>) will never return since it differs in the 2nd argument from the success typing arguments: (number(),number())

** (Mix) Dialyzer reported 2 warnings


: 1

- できた。
    - ちなみにここまで単純なやつはそもそもcompile時点でwarningが出る。
- [先程のLUSE記事](http://learnyousomeerlang.com/dialyzer)では、どのようなコードだと警告されて、どのようなコードは見逃されるか、順を追って説明している。
- 読め！ で終わりといえば終わりなのだが、ここでは実際にDialyzerを使いながらコードを書いていての印象・実感ベースで、「いい感じのElixirコードを書くということはどういうことなのか」について最近思っていることを書きたい。

## Conclusion

- Typespecsを書く。そこから始めよう。
- 型安全な言語でのプログラミングに一度触れてみるのオススメ。

## Code Quality

- コードの質についてはいろいろな評価軸があるので、おいそれと適当なことは言えない。
- が、あえて幾つかあげてみる。
    - そもそも
        - 動く（クラッシュしない）
        - 動いたり動かなかったりする（条件によってクラッシュする可能性がある）
        - 動かない、、、のは問題外
    - 改良や機能追加、バグ修正が
        - 容易である（動いていたものが動かなくなりにくい、修正作業がし易い）
        - 容易でない（取っ掛かりが掴みづらい、作用の影響が局所化されていない）
    - レビュアーや初見者が
        - 読みやすい
        - 読みにくい

- クラッシュせずに動き、修正が容易で、読みやすいコードに携われることは幸福であるように思われる。
- そのようないい感じのコードはElixirではどうすれば書けるだろうか。

## Proposal: Let Dialyzer Work

- ここでタイトルの提言をしたい。つまり「Dialyzerに仕事をさせよ」。

- ただし注意：CIや手元環境で定期的にDialyzerを回す**だけ**では半分くらいしか意味がないと思っている。
- 重要なのは
    - 「Dialyzerは仕事ができないことがある」という前提を持っておくことと、
    - 「Dialyzerが満足な仕事をできるようなコードを**書いて差し上げる**」という謙虚な姿勢である。

## Dialyzer May Not Work

- [先程のLUSE記事](http://learnyousomeerlang.com/dialyzer)に書かれている通り、Erlangのような言語に対してできる型解析は限られている。
- Dialyzerは基本的にはコードに問題はないものと前提を置きながら、徐々に条件を狭めていき、最終的に**絶対おかしいとわかったとき**以外は口を挟んでこない。

- 記事中のごく最初の方の例として、以下のコードがある。Elixirで書き換えて例として試してみる。
    - そのままだと`fetch/0`の実装が不足しているので、`fetch/1`に変更したが、`fetch/1`が`1`以外を返した場合にクラッシュする状況は同じ。
    - `main/1`から変数を取れるようになっているところがポイントで、Dialyzerの結果に影響する

In [1]:
defmodule Dial.Impl do
  def main(term) do
    x =
      case fetch(term) do
        1 -> :some_atom
        2 -> 3.14
      end
    convert(x)
  end

  defp convert(x) when is_atom(x), do: {:atom, x}

  defp fetch(term), do: term
end

Dial.Impl.main(1)

{:atom, :some_atom}

In [2]:
Dial.Impl.main(2)

FunctionClauseError: 1

- ファイルに定義してDialyzeしてみると、

In [1]:
# Bash
cd 20170913_let_dialyzer_work/dial
mix dialyze

Finding applications for analysis
Compiling 1 file (.ex)
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 2 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt


- （`Dial`と`Dial.Impl`で2モジュール）
- 警告無しで通ってしまう。
- 次に以下のように、`Dial`からクラッシュするはずの値で呼び出してみる。

In [2]:
defmodule Dial do
  def run() do
    Dial.Impl.main(:init_value)
  end
end

Dial.run()

CaseClauseError: 1

- この状態で再度Dialyzeしてみる。

In [1]:
# Bash
cd 20170913_let_dialyzer_work/dial
mix dialyze

Finding applications for analysis
Compiling 1 file (.ex)
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 2 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt


- やっぱり通る。
- この場合、Dialyzerに何を警告して頂けたら嬉しいだろうか。
    - `convert/1`に`atom`以外が渡ってきたら`FunctionClauseError`になること？
    - `fetch/1`が`1`と`2`以外を返すかもしれないこと（`CaseClauseError`になりうること）？
        - 原文の`fetch/0`でもこれは同じ。
- しかしDialyzerは「絶対におかしい、必ずクラッシュする」ところまで推論に成功しない限りは何も言わないため、これらは一見するより難しい。
- ロジックはそのままで、Dialyzerに導いてもらうコツはないか。

## Typespecs

- そこで、「この関数は一応、こういう値を受け取ってこういう値を返すことを想定してるんでごぜえますが・・・」、と恐れ多くも意見することができるアノテーション機能が用意されている
- それがType Specifications (Typespecs)。
- Typespeces自体はロジックには影響を与えない。
- 試しに以下のように書いてみる。

In [1]:
defmodule Dial.Impl do
  @spec main(integer) :: {:atom, atom | float}
  def main(term) do
    x =
      case fetch(term) do
        1 -> :some_atom
        2 -> 3.14
      end
    convert(x)
  end

  defp convert(x) when is_atom(x), do: {:atom, x}

  defp fetch(term), do: term
end

Dial.Impl.main(1)

{:atom, :some_atom}

- `main/1`が今のところ正しく処理できるのは`integer`だけであると表明してみた。
- Elixirでデフォルト使用可能なtypeは[ここ](https://hexdocs.pm/elixir/typespecs.html)にまとまっている
- 相変わらず`1`なら問題ないが、その他の値ならクラッシュする。何も成長していない。
- このコードをdialyzeしてもらうと。。。

In [2]:
# Bash
cd 20170913_let_dialyzer_work/dial
mix dialyze

Finding applications for analysis
Compiling 1 file (.ex)
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 2 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
lib/dial.ex:2: Function run/0 has no local return

lib/dial.ex:3: The call 'Elixir.Dial.Impl':main('init_value') breaks the contract (integer()) -> {'atom',atom() | float()}

** (Mix) Dialyzer reported 2 warnings


: 1

- 無事（？）怒ってもらえた。
- "Breaking of type contract"はtypespecsを導入することで検出できる（**ことがある**）違反の一つ。
    - この場合、`:init_value`というatomは`Dial.Impl.main/1`が表明しているtypespecesとマッチしないので、「契約違反」だというのである。

- では`integer`だがクラッシュする値だとどうだろうか。

In [2]:
defmodule Dial do
  def run() do
    Dial.Impl.main(2)
  end
end
    
Dial.run()

FunctionClauseError: 1

In [1]:
# Bash
cd 20170913_let_dialyzer_work/dial
mix dialyze

Finding applications for analysis
Compiling 1 file (.ex)
Finding suitable PLTs
Looking up modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding applications for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Checking 399 modules in dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt
Finding modules for analysis
Analysing 2 modules with dialyze_erlang-20.0_elixir-1.5.1_deps-dev.plt


- こちらは通ってしまう。
- 他にも色々試してみるとわかるが、ソースコード上に現れている実際の関数呼び出しから、クラッシュする値が予期せず指定されていないかどうか検査、といったことは残念ながらDialyzerにはそこまで期待はできない。
    - 基本的には値がまともかまでは関知しない。
    - Success type analysisがうまくハマると、何らかの「確実に落ちる」ケースを見つけてくれる**こともある**。
    - 定数を返す関数に依存している部分に関しては、余計な分岐があれば警告してくれる。原文のまま、`fetch/0`に適当な実装を与えて試してみるとわかる。
- Type contract違反（spec側が間違っているケース含む）については、感覚として引数の違反は発見してくれるケースが多い。
  一方、返り値の違反は場合による。
     - やっぱりSuccess type analysisがうまくハマると見つけてくれる**こともある**が、いつもそんなにはうまく行かない。

- 本当は、ここまでのサンプルコードをいじくり回して、うまくDialyzerがバグを見つけてくれるケースと駄目なケースについて、もう少し絞り込んでみようと思っていた。
    - Success type analysisの謎がかなり多いので、知見を増やすにはもうちょっと勉強しなければならない。
- が、上ページで書いたとおり一筋縄ではいかない。Dialyzerは**そんなに我々が思うようには働いてくれない**。
- にも関わらずなぜ「Let Dialyzer Work」を推すのか？

## Well-Typed Elixir Code For Well-Being

- **我々の幸福のためにほかならない**。
- 少なくともTypespecsを書いていくことで、それに違反するコードを書いてしまったとき（あるいはTypespecsを間違えてしまったとき）はそこそこの割合でType contract違反を起こしてもらえるようになっていく。
    - これによってバグを発見できる**可能性もある**。ただし過度な期待は禁物。
    - コードを修正する際、付随して修正が必要な場所や、意図せず影響を与えてしまった箇所を発見しやすくなる。こちらは比較的恩恵を得られるケースも多いという感覚がある。
    - 実装時の精神的な安心・自信につながる。

- **Typespecsはドキュメントの役割も果たす**ので、正しく書いていればdialyzeしなくとも意義がある。
    - つまり読みやすさの面でもプラスに働くため、一石二鳥。書くだけ得。加点法。
- Typespecsは関数に付与していくものなので、Typespecsを意識するということは自然と関数の姿を意識することに直結する。
    - 具体的には、「何を受け取って」、「何を返すのか」。
    - 自然にguardやpattern-matchによる分岐処理の必要性・必然性に至る。
    - いつの間にか「予期せぬ入力・出力」といったものが淘汰されていく。（処理されるべき値はすべて処理されていく）
    - 結果、クラッシュしないコードに近づく。

- Typespecsを書くということはDialyzerにうまく働いて頂くための一手段でしかないし、書いたからといってあらゆる災厄の種が尽きるわけではないが、
    - すぐにでも手を付けられて、
    - 目につくところから始められ、
    - 損がほとんど無い。
   
- しかしこう思うかもしれない：
    - せっかく型指定をしなくても動的に解決してくれて動いてくれる言語なのに、なぜ型を書いたり、型を意識しなければならないのか？ 面倒じゃないか？ 言語の特長をspoilしているのでは？

## Long-Term Coding Happiness

- 「クラッシュせずに動き、修正が容易で、読みやすいコード」を書きたいし、書き残したい。それが幸福であるように思うなら、それを長期間に渡って維持できることは至福であるはず。
- 自分の書いたコードをTypeによって守ることはそれにつながるので、十分に投資する価値がある。
    - 多少の面倒は慣れと[croma](https://github.com/skirino/croma)でカバーすれば良い。
    - 何かの機会で数ヶ月前のコードに立ち戻ることになったとき、Typespecsが充実していて、久しぶりの作業の後の不安がdialyzeによって解決するのは幸福である。

- 更に進むと、dialyzeによって**問題が報告される方が嬉しい**とかいう精神状態になってくる。
    - さんざ書いてきたように、Dialyzerの能力は我々の期待と全く一致しないので、Dialyzerが何も言わないということは「絶対にクラッシュする、ということはない」という程度しか保証しないからだ。
    - 一旦何か問題を報告してもらって、それを修正した上で問題がなくなったように見えた場合のほうが不思議と安心する。もっとも、これは単なる認知バイアスでしかないのでヤベー精神状態ではある。

- 一方、ちゃんとした静的型付け言語は、Dialyzerがごくごく部分的に保証する安全性を、より厳密な形で保証してくれる（[型安全性](https://www.wikiwand.com/en/Type_safety)）シロモノであるため、もっと安心感がある。
    - そもそも、安心とか幸福とかいうアレゲな尺度でこのポエムを書いている理由の多くを占めるのが、[Elm](http://elm-lang.org)でコンパイラに導かれたハッピーなコーディングに触れてしまったため。
- Elmでそこそこの量のコードを書き、コンパイルできているならば書いたとおり必ず動く世界観に浸ると、Elixirのtype-unsafeな世界はひどく心もとなく感じられてくる。コンパイルは通っても例外がありうるなんて！

- するとどうだろう。Elixirコードを書いていると、気づけばテストを書きたくなっている自分がいる！
    - Typespecsをかっちり書いても実行してみるまでは本当に安全かどうかわからないとき、人はテストに救いを求める。
    - 型安全でない言語では、Testは安心と幸福のために不可欠な存在になる。
        - もちろん型安全な言語でもTestで安心を得たい部分は存在するし、型の話抜きにしてもテストで挙動を確認するのは必要である。
        - 型安全でない言語では、Testに安心を求めなければならない部分がトレードオフとして必然的に広くなる。
    - なにより、実装の改善・修正（リファクタリング）のしやすさ、進めやすさにダイレクトに効いてくる。
- つまりTypespecsを書いてみることから出発して、なにか型安全な言語も書いてみたりすると、Testの重要性も再認識でき、長期にわたる幸福なコーディングにつながる

- これは個人的な印象だが、ErlangVMの恩恵に預かれるElixirでは、Typespecs/DialyzerおよびTestをそこに組み合わせることで、型安全な言語の安心感にある程度近づけると思っている。
    - もちろんTestは静的解析ではないので、実行時例外を経由してバグを検出する手段になる。
- そもそも、型安全だからといってクラッシュしないわけではない。どんな言語でも精妙なバグはあるし、クラッシュは発生する。Elmでも起こる。
    - そうなると、言語ごとにいざクラッシュが起きてしまった場合の機構がどうなっているか、が最終的な安心・幸福のため重要になる。
    - ErlangVMの実行モデルはこの点が優秀で、実行時例外は基本的には局所化されている。
    - すると、大局的には安全な環境で例外を起こしながらコードへのフィードバックを行い、TypespecsやTestを充実させることで一つ一つクラッシュの種を潰していき、クラッシュしないプログラムに近づく、というループが自然に回せる。
        - Erlangにおける"Let It Crash"の真の意味はこれだと感じている。

## For Ease Of Our Mind

- 安心し、自信を持ってコードを書けることは幸福だと思う。
- Elixirでそういった境地を目指すのであれば、まずはTypespecsを書くところから始めてみてはどうでしょうか。Dialyzerが応えてくれるやも。
- そしてElmも書いてみては。